# Туториал по использованию предобученной модели RuBERT в задаче классификации твитов по тональности

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers

I0214 14:12:11.676655  3532 file_utils.py:38] PyTorch version 1.4.0 available.
C:\Users\dmbot\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# загружаем 5000 позитивных и негативных твитов

df_tweets = pd.read_csv('./data/tweets.csv')

In [3]:
# создаем токенайзер для модели BERT, для его инициализации достаточно указать словарь, на котором обучалась предобученная модель
# BERT использует собственную токенизацию, никакой предобработки 

tokenizer = ppb.BertTokenizer(vocab_file='./Rubert/vocab.txt')

In [4]:
# токенизируем текст каждого твита, для BERT не требуется никакая дополнительная предобработка, лемматизация и прочее

tokenized = df_tweets['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)) )

In [5]:
# Пример токенизации текста: на входе - текст, а на выходе имеем массив с номерами токенов из словаря модели BERT

print(df_tweets['text'][0])
print(tokenized[0])
print(tokenizer.tokenize(df_tweets['text'][0]))

@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)
[101, 168, 10934, 230, 11147, 241, 1985, 358, 322, 54198, 128, 876, 37203, 128, 352, 1159, 822, 940, 2591, 156, 238, 7580, 77982, 1929, 934, 323, 7020, 2360, 122, 102]
['@', 'first', '_', 'time', '##e', 'хоть', 'я', 'и', 'школота', ',', 'но', 'поверь', ',', 'у', 'нас', 'то', 'же', 'самое', ':', 'd', 'общество', 'профили', '##рую', '##щи', '##и', 'предмет', 'типа', ')']


In [11]:
# инициализируем предобученную модель RuBERT из файла, 
# в json-файле конфигурации описаны параметры модели

config = ppb.BertConfig.from_json_file('./Rubert/bert_config.json')
model = ppb.BertModel.from_pretrained('./Rubert/rubert_model.bin', config = config)

I0214 14:16:12.220250  3532 modeling_utils.py:456] loading weights file ./Rubert/rubert_model.bin


In [7]:
# из-за того, что каждый твит в датасете имеет разную длину (количество токенов)
# мы делаем паддинг - заполнение нулями каждого массива токенов до длины максимального массива
# чтобы на выходе получить матрицу из токенизированных текстов одной длины

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [8]:
# посмотрим на размерность матрицы токенизированных твитов после паддинга

np.array(padded).shape

(5000, 133)

In [9]:
# Накладываем маску на значимые токены
# В данном случае нам важны все слова кроме нулевых токенов, появившихся на предыдущем шаге паддинга

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5000, 133)

In [39]:
# а теперь сформируем вектора текстов с помощью модели RuBERT

# это не быстрый процесс, импортируем инструмент для визуализации времени обработки в цикле
from tqdm import notebook

# для того, чтобы модель отработала в условиях ограниченных ресурсов - оперативной памяти, мы разделяем входной датасет на батчи.
# при батче в 100 твитов потребление оперативной памяти укладывается в 1Гб
batch_size = 100

# Делаем пустой список для хранения эмбеддингов (векторов) твитов
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        # преобразуем батч с токенизированными твитами в тензор 
        # по сути тензор - это многомерный массив, который может быть обработан нейронной сетью
        input_ids = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        
        # создаем тензор и для подготовленной маски
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        # передаем в модель BERT тензор из твитов и маску - на выходе получаем эмбеддинги - вектор текста твита
        # torch.no_grad() - для ускорения инференса модели отключим рассчет градиентов
        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask_batch)
        
        # в итоге собираем все эмбеддинги твитов в features
        embeddings.append(last_hidden_states[0][:,0,:].numpy())



A Jupyter Widget

KeyboardInterrupt: 

In [33]:
# преобразуем список батчей эмбеддингов в numpy-матрицу 
features = np.concatenate(embeddings)

In [32]:
# выводим размерность полученной матрицы эмбеддингов
# данная модель BERT формирует вектора текстов в 768-мерном пространстве признаков
features.shape

(300, 768)

In [23]:
# Выводим пример эмбеддинга для твита - осторожно много цифр!

print(df_tweets['text'][0])
print(features[0])

@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)
[-2.71498978e-01 -5.59076667e-01 -2.80984223e-01  2.64940172e-01
 -2.54880697e-01  2.69019127e-01 -4.17226553e-01  6.90964520e-01
  9.22523886e-02 -1.33143395e-01 -5.13912886e-02 -9.64237005e-02
 -3.92212003e-01  4.07325059e-01  4.54640687e-01 -1.16573918e+00
  3.71884644e-01  3.66923481e-01  6.32774413e-01 -2.60575920e-01
  6.80139720e-01 -1.35592461e-01 -1.66186810e-01 -7.37247229e-01
  1.22717768e-01 -6.80439472e-01 -9.33969021e-01 -2.73729622e-01
  1.24252713e+00  6.58041298e-01 -4.60716672e-02  1.58906430e-01
 -9.72143590e-01  1.04688776e+00 -7.93715894e-01 -8.88676047e-02
 -1.34215206e-01 -9.63649809e-01  7.61462674e-02 -4.12987471e-02
 -6.02056459e-02  1.20653138e-01  7.70601630e-01 -4.12695259e-01
 -4.21046257e-01  5.07617950e-01  6.47019267e-01 -2.52290487e-01
  7.04175889e-01  5.27700543e-01  1.02272320e+00 -6.13514543e-01
  6.23782277e-01 -7.47733116e-01  1.22249615e+00 -2.159

In [24]:
# Импортируем необходимые библиотеки для обучения классификатора на logreg и оценки качества

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [25]:
# Сохраним целевую переменную: метку тональности позитив/негатив

labels = df_tweets['positive']

In [26]:
# Разделяем матрицу признаков и целевую переменную на обучающий и тестовый набор

train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [27]:
# обучаем классификатор на основе логистической регрессии

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [39]:
# делаем пробное предсказание
tweet_index = 555

print('Text: ' + df_tweets['text'][tweet_index])
print('Predict label: ', lr_clf.predict(features[tweet_index:tweet_index+1][:])[0])
print('True label: ', df_tweets['positive'][tweet_index])


Text: @sergeylazarev С премьерой клипа,Серёж! Свои впечатления я уже написала.Клип сильнейший! И спасибо за эфир.Подняли настроение,как всегда.))
Predict label:  1
True label:  1


In [55]:
# оцениваем accuracy на тестовой выборке

lr_clf.score(test_features, test_labels)

0.964